# Testing the DEB model
## Santruckova et al. (2004)
1. Importing libraries

In [1]:
import numpy as np
import pandas as pd
from scipy.integrate import odeint
from scipy.optimize import dual_annealing
from scipy.optimize import differential_evolution

2. Defining DEB model

In [2]:
def DEBmodel (y, t, pars):
    #define initial pools (eu is assumed to be zero)
    Sl=y[0];    el=y[1];    X1l=y[2];     CO2l=y[3];
    X1u=y[4]
    #define parameters
    yA=pars[0]; 
    Km=pars[1];     
    v=pars[2];
    m=pars[3]; 
    g=pars[4]; 
    ce=pars[5];
    MX1=ce/4;
    #Scaling function for substrate uptake
    f=Sl/(Km+Sl) #labelled substrate only
    
    #Isotope signals
    eatm = 1
    X1atm = X1l/(X1l + X1u)
    
    #Fluxes
    uptake=(v*ce/yA)*(X1l+X1u)*f #labelled substrate only
    growth = (v*el-m*g)/(el + g)
    
    #Define derivatives
    ##Labelled pools
    dSldt = -uptake
    deldt = v*(f - el)
    dX1ldt = max(0, (X1l + X1u)*growth) + min(0, (X1l + X1u)*growth*X1atm) 
    dCO2ldt = uptake*(1 - yA) + ce*((X1l + X1u)*el*(v-growth)) - max(0, (X1l + X1u)*growth)*MX1 - min(0, (X1l + X1u)*growth*X1atm)*MX1 
    ##Unabelled pools
    #dSudt
    #deudt = - v*eu
    dX1udt = min(0, (X1l + X1u)*growth*(1-X1atm)) 
        
    return dSldt, deldt, dX1ldt, dCO2ldt, dX1udt;

3. Defining outputs from DEB model

In [3]:
def calcDEB (model, pars, t, y0):
    #model parameters
    ##yA, Km, v, m, g, ce
    pars_model=pars[0:6]
    #conversion factors
    ##ce, nX1
    conversions=pars[5:7]
    
    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))
    #calculate labelled biomass (Bl), kec factor, and labelled chloroform flush (Flush14C) 
    Bl=0.25*conversions[0]*y[:, 2] + conversions[0]*y[:, 1]*(y[:, 2] + y[:, 4])
    kec = (0.25*conversions[1] + y[:, 1])/(0.25 + y[:, 1])
    Flush14C = Bl*kec
        
    #Create data with predictions
    yhat = np.concatenate((y[:, 0].reshape(len(d.Time),1), #Glucose
                           y[:, 3].reshape(len(d.Time),1),#CO2
                           kec.reshape(len(d.Time),1),
                           Flush14C.reshape(len(d.Time),1)), axis=1)
    
    return yhat

4. Defining objective function

In [4]:
def obj_funDEB (x):
    #define parameters
    ##yA, Km, v, m, g, ce, nX1
    pars = x
    #initial conditions
    Sl_i = d.Sinit[0]
    # el_i = 0, Xl_i = 0, CO2l = 0, CO2u = 0, eu = 0  
    X1u_i = d.Cmicinit[0]/(pars[5]*(pars[6]/4))
    
    y0 = np.array([Sl_i, 0, 0, 0,X1u_i])
    #times
    t = d.Time
    #model simulations
    yhat_full = calcDEB(DEBmodel, pars, t, y0)
    #observations
    obs=np.concatenate((np.array([d.S]).reshape(len(d.Time),1),
                        np.array([d.CO214cumul]).reshape(len(d.Time),1),
                        np.array([d.kec]).reshape(len(d.Time),1),
                        np.array([d.Cmic14]).reshape(len(d.Time),1)), axis=1)
    #weights
    weights=np.concatenate((np.nanmean(d.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(d.CO214cumul).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(d.kec/5).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean((d.Cmic14)).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    out=np.nansum(((yhat_full-obs)/weights)**2)
    return out

5. Calculate the goodness of fit

In [5]:
def goodnessDEB (x):
    #define parameters
    ##Km, v, m, g, ce, nX1
    pars = x
    #initial conditions
    Sl_i = d.Sinit[0]
    # el_i = 0, Xl_i = 0, CO2l = 0, CO2u = 0, eu = 0  
    X1u_i = d.Cmicinit[0]/(pars[5]*(pars[6]/4))
    
    y0 = np.array([Sl_i, 0, 0, 0,X1u_i])
    #times
    t = d.Time
    #model simulations
    yhat_full = calcDEB(DEBmodel, pars, t, y0)
    
    #observations
    obs=np.concatenate((np.array([d.S]).reshape(len(d.Time),1),
                        np.array([d.CO214cumul]).reshape(len(d.Time),1),
                        np.array([d.kec]).reshape(len(d.Time),1),
                        np.array([d.Cmic14]).reshape(len(d.Time),1)), axis=1)
    
    R2=1-np.nansum((obs-yhat_full)**2)/np.nansum((obs-np.nanmean(obs))**2)
    ll=-np.nansum((obs-yhat_full)**2)/2/np.nanstd(obs)**2
    AIC = len(pars)*2 - 2*ll
    out = np.array([R2, ll, AIC])
    return out

Following function return the model solution for visualization in R

In [6]:
def predDEB (model, pars, t, y0):
    #model parameters
    ##yA, Km, v, m, g, ce
    pars_model=pars[0:6]
    #conversion factors
    ##ce, nX1
    conversions=pars[5:7]
    
    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))
    #calculate labelled biomass (Bl), kec factor, and labelled chloroform flush (Flush14C) 
    Bl=0.25*conversions[0]*y[:, 2] + conversions[0]*y[:, 1]*(y[:, 2] + y[:, 4])
    kec = (0.25*conversions[1] + y[:, 1])/(0.25 + y[:, 1])
    Flush14C = Bl*kec
    
    #Create data with predictions
    yhat = np.concatenate((y[:, 0].reshape(61,1), #Glucose
                           y[:, 3].reshape(61,1),#CO2
                           kec.reshape(61,1),
                           Flush14C.reshape(61,1)), axis=1)
    
    return yhat

Reading data

In [7]:
d = pd.read_csv('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/SoilMBVariabilityData/Santruckova2004.csv', sep=',')
print(d)

                      Study        Soil Substrate  Clay    pH  Ctot  Ntot  \
0  Santruckova et al., 2004  sandy loam   Glucose    20  5.47  1.37  0.15   
1  Santruckova et al., 2004  sandy loam   Glucose    20  5.47  1.37  0.15   
2  Santruckova et al., 2004  sandy loam   Glucose    20  5.47  1.37  0.15   
3  Santruckova et al., 2004  sandy loam   Glucose    20  5.47  1.37  0.15   
4  Santruckova et al., 2004  sandy loam   Glucose    20  5.47  1.37  0.15   
5  Santruckova et al., 2004  sandy loam   Glucose    20  5.47  1.37  0.15   

       Time  Cmicinit      Sinit     Cmic12    Cmic14     CO214  CO214cumul  \
0  0.000000  7.694132  26.376234   7.694132  0.000000  0.000000    0.000000   
1  0.333333  7.694132  26.376234   8.904390  3.680103  5.101495    1.700498   
2  0.666667  7.694132  26.376234  10.150397  4.562771  5.468979    3.523491   
3  1.000000  7.694132  26.376234  11.714008  4.819833  5.624263    5.398246   
4  2.000000  7.694132  26.376234  10.586995  4.428162  1.129519  

Estimating parameters

In [8]:
Santruckova2004DA=dual_annealing(obj_funDEB, [(0.05, 1), #yA
                                              (10, 1000), #Km
                                              (0.001, 20), #v
                                              (1e-12, 0.1), #m
                                              (0.1, 3), #g
                                              (0.1, 10), #ce
                                              (0, 1)]) #nX1
                                              

In [9]:
print(Santruckova2004DA)
print(goodnessDEB(Santruckova2004DA.x))

     fun: 0.29734016350393594
 message: ['Maximum number of iteration reached']
    nfev: 16425
    nhev: 0
     nit: 1000
    njev: 303
  status: 0
 success: True
       x: array([9.94919283e-01, 3.04134402e+02, 4.35400564e+00, 1.35499232e-03,
       3.33754748e-01, 8.75915087e+00, 2.22208743e-01])
[ 0.98600717 -0.15392108 14.30784216]


In [10]:
Santruckova2004DE=differential_evolution(obj_funDEB, [(0.05, 1), #yA
                                              (10, 1000), #Km
                                              (0.001, 20), #v
                                              (1e-12, 0.1), #m
                                              (0.1, 3), #g
                                              (0.1, 10), #ce
                                              (0, 1)]) #nX1

In [11]:
print(Santruckova2004DE)
print(goodnessDEB(Santruckova2004DE.x))

     fun: 0.29734160597460435
     jac: array([ 1.58284496e-04, -1.19237858e-05,  1.56569203e-04, -2.09393614e-03,
       -2.04369854e-04,  2.10942376e-07, -7.37743200e-04])
 message: 'Optimization terminated successfully.'
    nfev: 9384
     nit: 87
 success: True
       x: array([9.94986979e-01, 3.03869160e+02, 4.35170354e+00, 1.34161573e-03,
       3.33805689e-01, 2.35939019e+00, 2.22188125e-01])
[ 0.98600431 -0.15395262 14.30790524]


In [13]:
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/Santruckova2004Pars.csv', Santruckova2004DE.x.reshape(1,7), delimiter=",")

Exporting solution for R

In [14]:
#initial conditions
Sl_i = d.Sinit[0]
X1u_i = d.Cmicinit[0]/(Santruckova2004DE.x[5]*(Santruckova2004DE.x[6]/4))
    
y0 = np.array([Sl_i, 0, 0, 0,X1u_i])

#times
t = np.arange(0, 3.05, 0.05)
#model simulations
Santruckova2004Pred = predDEB(DEBmodel, Santruckova2004DE.x, t, y0)
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/Santruckova2004Pred.csv', Santruckova2004Pred, delimiter=",")
print(Santruckova2004Pred)

[[2.63762341e+01 0.00000000e+00 2.22188125e-01 0.00000000e+00]
 [2.40529523e+01 7.18667518e-02 2.66077039e-01 5.99049826e-01]
 [2.19020578e+01 2.33687945e-01 2.94805249e-01 1.25011822e+00]
 [1.99025312e+01 4.58265332e-01 3.13534694e-01 1.88604837e+00]
 [1.80408871e+01 7.27061341e-01 3.25347464e-01 2.47533642e+00]
 [1.63081828e+01 1.02664021e+00 3.32208176e-01 3.00363068e+00]
 [1.46981709e+01 1.34677322e+00 3.35437367e-01 3.46550071e+00]
 [1.32061086e+01 1.67937611e+00 3.35962913e-01 3.86046565e+00]
 [1.18279729e+01 2.01789066e+00 3.34461860e-01 4.19093106e+00]
 [1.05599459e+01 2.35691462e+00 3.31444338e-01 4.46103318e+00]
 [9.39809045e+00 2.69197413e+00 3.27305154e-01 4.67593692e+00]
 [8.33816409e+00 3.01937892e+00 3.22356497e-01 4.84137268e+00]
 [7.37553564e+00 3.33612509e+00 3.16849206e-01 4.96330766e+00]
 [6.50517691e+00 3.63982396e+00 3.10986906e-01 5.04770103e+00]
 [5.72170712e+00 3.92864430e+00 3.04935647e-01 5.10031789e+00]
 [5.01947157e+00 4.20126003e+00 2.98830643e-01 5.126589